In [2]:
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import Augmentor
import os
import cv2
from pascal_voc_writer import Writer
import shutil
import numpy as np

import random
from random import randrange

import numpy
def find_coeffs(source_coords, target_coords):
    matrix = []
    for s, t in zip(source_coords, target_coords):
        matrix.append([t[0], t[1], 1, 0, 0, 0, -s[0]*t[0], -s[0]*t[1]])
        matrix.append([0, 0, 0, t[0], t[1], 1, -s[1]*t[0], -s[1]*t[1]])
    A = numpy.matrix(matrix, dtype=numpy.float)
    B = numpy.array(source_coords).reshape(8)
    res = numpy.dot(numpy.linalg.inv(A.T * A) * A.T, B)
    return numpy.array(res).reshape(8)

In [6]:
#Create Fixed Sized Random 3-LETTER Plates

lets = 'ABCDEFGHIJKMNOPQRSTUVWXYZ'
num = '0123456789'
total_plates_to_make = 1000

for x in range(total_plates_to_make):
    l1 = random.choice(lets)
    l2 = random.choice(lets)
    l3 = random.choice(lets)
    n1 = random.choice(num)
    n2 = random.choice(num)
    n3 = random.choice(num)
    n4 = random.choice(num)


    font = ImageFont.truetype("Sauerkrauto W00 Regular.ttf",58)
    font_num = ImageFont.truetype("Uniform Extra Condensed.ttf",35)
    img=Image.open('template.png')
    draw = ImageDraw.Draw(img)
    draw.text((68, -10),"L",(0,0,0),font=font)
    draw.text((97, -10),l2,(0,0,0),font=font)
    draw.text((126, -10),l3,(0,0,0),font=font)

    draw.text((52, 40),n1,(0,0,0),font=font)
    draw.text((81, 40),n2,(0,0,0),font=font)
    draw.text((110, 40),n3,(0,0,0),font=font)
    draw.text((139, 40),n4,(0,0,0),font=font)

    draw.text((185, 17),n2,(0,0,0),font=font_num)
    draw.text((198, 17),n4,(0,0,0),font=font_num)
    draw = ImageDraw.Draw(img)
    img.save('random_plates/L'+str(l2)+str(l3)+str(n2)+str(n4)+str(n1)+str(n2)+str(n3)+str(n4)+".png")

In [7]:

#Create Fixed Sized Random 2-LETTER Plates    
for x in range(int(total_plates_to_make * 0.5)):
    l1 = random.choice(lets)
    l2 = random.choice(lets)
    l3 = random.choice(lets)
    n1 = random.choice(num)
    n2 = random.choice(num)
    n3 = random.choice(num)
    n4 = random.choice(num)


    font = ImageFont.truetype("Sauerkrauto W00 Regular.ttf",58)
    font_num = ImageFont.truetype("Uniform Extra Condensed.ttf",35)
    img=Image.open('template.png')
    draw = ImageDraw.Draw(img)
    draw.text((80, -10),"L",(0,0,0),font=font)
    draw.text((109, -10),l2,(0,0,0),font=font)

    draw.text((64, 40),n1,(0,0,0),font=font)
    draw.text((93, 40),n2,(0,0,0),font=font)
    draw.text((122, 40),n3,(0,0,0),font=font)

    draw.text((185, 17),n2,(0,0,0),font=font_num)
    draw.text((198, 17),n4,(0,0,0),font=font_num)
    draw = ImageDraw.Draw(img)
    img.save('random_plates/L'+str(l2)+str(n2)+str(n4)+str(n1)+str(n2)+str(n3)+".png")

In [6]:
# Prospective augment and paste randomly on backgrounds 

bg_path = './bgs/'
augmented_img_path = './random_plates/'
output_path = './plate_with_bgs/'
os.makedirs(output_path,exist_ok=True)

lbl_path = './plate_with_bgs_lbls/'
os.makedirs(lbl_path,exist_ok=True)

# tif_img_path = './img_tif/'
# tif_lbl_path = './lbl_tif/'
#os.makedirs(tif_img_path,exist_ok=True)
#os.makedirs(tif_lbl_path,exist_ok=True)

test_annot = False
image_resize = 600

for i,x in enumerate(os.listdir(augmented_img_path)):
    if True:#x.endswith('LMC917921.png'):
        
        background = Image.open(bg_path+random.choice(os.listdir(bg_path))).convert("RGBA")
        img = Image.open(augmented_img_path+x).convert("RGBA")
            
        background = background.resize((image_resize,image_resize), Image.ANTIALIAS)
        w,h = background.size

        if i%2==0:
            top_left=(random.randint(0,50), random.randint(120,140))
            top_right=(random.randint(120,140), random.randint(0,100))
            bot_right=(random.randint(100,140), random.randint(150,300))
            bot_left=(random.randint(10,20), random.randint(200,250))
        else:
            top_left=(random.randint(130,150), random.randint(100,120))
            top_right=(random.randint(300,330), random.randint(130,140))
            bot_right=(random.randint(300,330), random.randint(200,220))
            bot_left=(random.randint(130,150), random.randint(200,260))


        if i%3==0:
            top_left=(0, 0)
            top_right=(217, 0)
            bot_right=(217, 112)
            bot_left=(0, 112)
            coeffs = find_coeffs([(0, 0), (217, 0), (217, 112), (0, 112)],[top_left, top_right, bot_right, bot_left])
            img =img.transform((500, 500), Image.PERSPECTIVE, coeffs,Image.BICUBIC,fill=0)
        else:
            coeffs = find_coeffs([(0, 0), (217, 0), (217, 112), (0, 112)],[top_left, top_right, bot_right, bot_left])
            img =img.transform((500, 500), Image.PERSPECTIVE, coeffs,Image.BICUBIC,fill=0)
#         img.save('test2.png')

        box_x = random.randint(1,70)
        box_y = random.randint(1,100)

        rand_scale=random.randint(0,1)
        if rand_scale == 1:
#                 print('scaled')
            scale = random.uniform(0.3,1)
            img2 = img.resize((int(img.size[0] * scale),int(img.size[1] * scale)), resample=0)

            xmin = min(int(top_left[0] * scale +box_x), int(bot_left[0] * scale + box_x))
            ymin = min(int(top_left[1] * scale +box_y), int(top_right[1] * scale+ box_y))
            xmax = max(int(top_right[0] * scale +box_x), int(bot_right[0] * scale + box_x))
            ymax = max(int(bot_left[1] * scale +box_y), int(bot_right[1] * scale + box_y))
            xmax = w if xmax > w else xmax
            ymax = h if ymax > h else ymax

        else:

            xmin = min(int(top_left[0] +box_x), int(bot_left[0]  + box_x))
            ymin = min(int(top_left[1] +box_y), int(top_right[1] + box_y))
            xmax = max(int(top_right[0] +box_x), int(bot_right[0] + box_x))
            ymax = max(int(bot_left[1] +box_y), int(bot_right[1] + box_y))
            xmax = w if xmax > w else xmax
            ymax = h if ymax > h else ymax

            img2 = img.copy()
        background.paste(img2, (box_x,box_y), img2)
        background.save(output_path+x.split('.')[0]+'_bg.png')
#         background.save(tif_img_path+x.split('.')[0]+'_bg.tif')

# writing pascal VOC formal XML annotations         
        writer = Writer(output_path+x.split('.')[0]+'_bg.png', w,h)
        writer.addObject('Number_Plate', xmin, ymin, xmax, ymax)
        writer.save(lbl_path+x.split('.')[0]+'_bg.xml')


######################################################################################################

# for masking algos (tiff labels)

#         data = np.array(img)
#         red, green, blue, alpha = data.T 

#         white_areas = (red >= 0) & (blue >= 0) & (green >= 0)
#         data[..., :-1][white_areas.T] = (255, 255, 255)

#         im2 = Image.fromarray(data)

#         background = np.zeros([background.size[0],background.size[1]],dtype=np.uint8)
#         background = Image.fromarray(background)
#         background.paste(im2, (box_x,box_y), im2)

#         ar = np.array(background)
#         ar[ar == 255] = 1
#         img_lbl = Image.fromarray(ar)
#         img_lbl.save(tif_lbl_path+x.split('.')[0]+'_bg.tif')

KeyboardInterrupt: 

In [7]:
if test_annot:
    import xml.etree.ElementTree as ET
    for xml_file in [f for f in os.listdir('./plate_with_bgs_lbls') if f.endswith(".xml")]:
        tree = ET.parse(os.path.join('./plate_with_bgs_lbls', xml_file))
        root = tree.getroot()

        file_name = None

        for elem in root:
            if elem.tag == 'filename':
                file_name = 'a'

        if elem.tag == 'object':
            obj_name = None
            coords = []
            for subelem in elem:
                if subelem.tag == 'name':
                    obj_name = subelem.text
                if subelem.tag == 'bndbox':
                    xmin1 = ""
                    xmax1 = ""
                    ymin1 = ""
                    ymax1 = ""
                    for subsubelem in subelem:
                        f = float(subsubelem.text)
                        i = int(f)
                        if subsubelem.tag == "xmin":
                            xmin1 = str(i)
                        if subsubelem.tag == "xmax":
                            xmax1 = str(i)
                        if subsubelem.tag == "ymin":
                            ymin1 = str(i)
                        if subsubelem.tag == "ymax":
                            ymax1 = str(i)

#                         fn = [x for x in os.listdir('./output_with_bgs/')]
        g = cv2.imread('./output_with_bgs/'+xml_file.split('.')[0]+'.png')

        strt = (int(xmin1),int(ymin1))
        stp = (int(xmax1),int(ymax1))
        gg = cv2.rectangle(g,strt,stp,(0,255,0),2)
        cv2.imshow('imgg',gg)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [9]:
## Making Train Val Splits

os.makedirs('./plates/train/images/',exist_ok=True)
os.makedirs('./plates/train/annotations/',exist_ok=True)
os.makedirs('./plates/validation/images/',exist_ok=True)
os.makedirs('./plates/validation/annotations/',exist_ok=True)
# os.makedirs('./plates/tif/imgs/',exist_ok=True)
# os.makedirs('./plates/tif/lbls/',exist_ok=True)


for x in range (0,50):
    img = random.choice(os.listdir('./plate_with_bgs'))
    img_name = img.split('.')[0]
    shutil.move('./plate_with_bgs/'+img,'./plates/validation/images/'+img)
    shutil.move('./plate_with_bgs_lbls/'+img_name+'.xml','./plates/validation/annotations/'+img_name+'.xml')
    

    
# for x in range (0,300):
#     img = random.choice(os.listdir('./img_tif/'))
#     img_name = img.split('.')[0]
#     shutil.move('./img_tif/'+img,'./plates/tif/imgs/'+img)
#     shutil.move('./lbl_tif/'+img_name+'.tif','./plates/tif/lbls/'+img_name+'.tif')
    
    
files_r = os.listdir('./output_with_bgs')
for f in files_r:
    shutil.move('./output_with_bgs/'+f, './plates/train/images/'+f)
            
lbls_r = os.listdir('./output_with_bgs_lbls/')
for f in lbls_r:
    shutil.move('./output_with_bgs_lbls/'+f, './plates/train/annotations/'+f)

FileNotFoundError: [Errno 2] No such file or directory: './plate_with_bgs_lbls/LCP401490_bg.xml'